# Tugas Besar 1 IF3270 Pembelajaran Mesin <br /> Feedforward Neural Network

## Kelompok 39

- Dzaky Satrio Nugroho - 13522059
- Julian Caleb Simandjuntak - 13522099
- Rafiki Prawhira Harianto - 13522065

In [20]:
# Import dulu
import numpy as np


In [21]:
# Fungsi Aktivasi 

class ActivationFunction:
    
    # Fungsi linear
    @staticmethod
    def linear(x: np.ndarray) -> np.ndarray:
        return x

    # Fungsi ReLU
    @staticmethod
    def relu(x: np.ndarray) -> np.ndarray:
        return np.maximum(0, x)

    # Fungsi Sigmoid
    @staticmethod
    def sigmoid(x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))

    # Fungsi Hyperbolic Tangent
    @staticmethod
    def tanh(x: np.ndarray) -> np.ndarray:
        return np.tanh(x)

    # Fungsi Softmax
    @staticmethod
    def softmax(x: np.ndarray) -> np.ndarray:
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    # Fungsi Leaky ReLU
    @staticmethod
    def leaky_relu(x: np.ndarray, alpha=0.1) -> np.ndarray:
        return np.maximum(alpha*x, x)

    # Fungsi Swish
    @staticmethod
    def swish(x: np.ndarray) -> np.ndarray:
        return x * ActivationFunction.sigmoid(x)

In [22]:
# Fungsi Loss

class LossFunction:
    
    # Mean Squared Error
    @staticmethod
    def mse(y_pred: np.ndarray, y_true: np.ndarray) -> float:
        mse = np.sum((y_pred - y_true) ** 2) / len(y_true)
        return mse

    # Binary Cross-Entropy
    @staticmethod
    def bce(y_pred: np.ndarray, y_true: np.ndarray) -> float:
        bce = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
        return bce

    # Categorical Cross-Entropy
    @staticmethod
    def cce(y_pred: np.ndarray, y_true: np.ndarray) -> float:
        cce = -1 / len(y_true) * np.sum(np.sum(y_true * np.log(y_pred)))
        return cce
    
    # @staticmethod
    # def calculate_loss(loss_type: str, y_pred: np.ndarray, y_true: np.ndarray) -> float:
    #     if loss_type == 'mse':
    #         return LossFunction.mse(y_pred, y_true)
    #     elif loss_type == 'bce':
    #         return LossFunction.bce(y_pred, y_true)
    #     elif loss_type == 'cce':
    #         return LossFunction.cce(y_pred, y_true)
    #     else:
    #         raise ValueError(f"Jenis loss tidak dikenal.")

In [23]:
# Turunan Fungsi Aktivasi

class ActivationFunctionDerivative:

    # Fungsi Linear
    @staticmethod
    def linear(x: np.ndarray) -> np.ndarray:
        return np.ones_like(x)
    
    # Fungsi RelU
    @staticmethod
    def relu(x: np.ndarray) -> np.ndarray:
        return np.where(x > 0, 1, 0)
    
    # Fungsi Sigmoid
    @staticmethod
    def sigmoid(x: np.ndarray) -> np.ndarray:
        sigmoidx = ActivationFunction.sigmoid(x)
        return sigmoidx * (1 - sigmoidx)
    
    # Fungsi Hyperbolic Tangent
    @staticmethod
    def tanh(x: np.ndarray) -> np.ndarray:
        return (2 / (2 * np.sinh(x))) ** 2

    # Fungsi Softmax
    @staticmethod
    def softmax(x: np.ndarray) -> np.ndarray:
        softmaxx = ActivationFunction.softmax(x)
        n = x.size
        matrix = []
        for i in range(1,n+1):
            row = []
            for j in range(1,n+1):
                row.append(softmaxx[i-1] * ((i == j) - softmaxx[j-1]))
            matrix.append(row)

        return np.array(matrix)

    # Fungsi Leaky ReLU
    @staticmethod
    def leaky_relu(x: np.ndarray, alpha=0.1) -> np.ndarray:
        return np.where(x > 0, 1, alpha)
    
    # Fungsi Swish
    @staticmethod
    def swish(x: np.ndarray) -> np.ndarray:
        sigmoidx = ActivationFunction.sigmoid(x)
        return sigmoidx * (1 + x - x * sigmoidx)

In [24]:
"""
Inisialisasi 1 layer bobot dengan parameter wajib shape yang merupakan tuple berisi ukuran matrix bobot
Contoh: 
shape=(3, 4) berarti:
- Untuk layer dengan 3 neuron awal dan layer dengan 4 neuron berikutnya
- Menghasilkan matrix bobot dengan 4 kolom berdasarkan bias + neuron layer awal dikali 4 kolom berdasarkan neuron layer berikutnya
"""
class WeightInitializer:    
    @staticmethod
    def zeros(shape):
        w = np.zeros((shape[1], shape[0]))
        b = np.zeros((shape[1], 1))
        return np.hstack((b, w))

    @staticmethod
    def uniform(shape, lower_bound=-0.1, upper_bound=0.1, seed=None):
        if seed is not None:
            np.random.seed(seed)
        w = np.random.uniform(lower_bound, upper_bound, (shape[1], shape[0]))
        b = np.random.uniform(lower_bound, upper_bound, (shape[1], 1))
        return np.hstack((b, w))

    @staticmethod
    def normal(shape, mean=0.0, variance=1.0, seed=None):
        if seed is not None:
            np.random.seed(seed)
        std_dev = np.sqrt(variance)  # Konversi variance ke standard deviation
        w = np.random.normal(mean, std_dev, (shape[1], shape[0]))
        b = np.random.normal(mean, std_dev, (shape[1], 1))
        return np.hstack((b, w))
    
    # @staticmethod
    # def initialize_weights(initialization_type: str, shape, bias=1, lower_bound=-0.1, upper_bound=0.1, mean=0.0, variance=1.0, seed=None):
    #     if initialization_type == 'zeros':
    #         return WeightInitializer.zeros(shape, bias=bias)
    #     elif initialization_type == 'uniform':
    #         return WeightInitializer.uniform(shape, bias=bias, lower_bound=lower_bound, upper_bound=upper_bound, seed=seed)
    #     elif initialization_type == 'normal':
    #         return WeightInitializer.normal(shape, bias=bias, mean=mean, variance=variance, seed=seed)
    #     else:
    #         raise ValueError(f"Jenis inisialisasi '{initialization_type}' tidak dikenal.")
    
# Contoh penggunaan
zero_weights = WeightInitializer.zeros((3,4))
uniform_weights = WeightInitializer.uniform((3,4))
normal_weights = WeightInitializer.normal((3,4))
print(zero_weights)
print(uniform_weights)
print(normal_weights)
# output:
# [[0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0.]]
# [[-0.01535893  0.01369034 -0.05323466  0.05658833 -0.0295206 ]
#  [-0.05550326  0.06834675  0.03335891  0.03058884  0.05339302]
#  [-0.07148145 -0.0300887  -0.07895016  0.05567438  0.03814023]]
# [[ 0.25158888  1.34267031 -0.03036228 -1.16132763 -0.66723239]
#  [ 0.64635165 -1.22173216 -0.41217616  0.9060914  -0.56040958]
#  [-0.01703997 -0.64723922  0.89749136 -0.05013735 -1.27893224]]

In [ ]:
# Mencoba membuat FFNN 

# Yang menjadi ketentuan parameter FFNN:
# - Jumlah layer
# - Jumlah neuron tiap layer
# - Fungsi aktivasi tiap layer
# - Fungsi loss dari model
# - Metode inisialisasi bobot

# Method FFNN:
# - Inisialisasi bobot
# - Menyimpan bobot
# - Menyimpan gradien bobot
# - Menampilkan model struktur jaringan, bobot, dan gradien
# - Menampilkan distribusi bobot
# - Menampilkan distribusi gradien bobot
# - Save and load
# - Forward propagation
# - Backward propagation
# - Weight update dengan gradient descent

# Parameter pelatihan FFNN:
# - Batch size
# - Learning rate
# - Jumlah epoch
# - Verbose


class FFNN:
    # Untuk sementara, di edit manual, bukan input.
    def __init__(self):
        # Parameter-parameter
        # Menerima jumlah neuron dari setiap layer (sekaligus jumlah layernya) termasuk input dan output
        self.layers = [3, 4, 2] # Contoh: [1, 2, 3]
        # Menerima fungsi aktivasi tiap layer
        self.activations = ["sigmoid", "sigmoid"] # Contoh: ["sigmoid", "relu"]
        # Menerima fungsi loss
        self.loss = "mse" # Contoh: "mse"
        # Menerima metode inisialisasi bobot
        self.initialization = "uniform" # Contoh: "zeros"
        # Jika bobot bukan zeros, menerima seeding
        self.seed = 0
        
        # Inisialisasi bias dan bobot, beserta gradiennya
        self.weights = []
        self.gradients_w = []
        
        for i in range(1, len(self.layers)):
            in_size, out_size = self.layers[i - 1], self.layers[i]
            if self.initialization == 'zeros':
                w = WeightInitializer.zeros((in_size, out_size))
            elif self.initialization == 'uniform':
                w = WeightInitializer.uniform((in_size, out_size), seed=self.seed)
            elif self.initialization == 'normal':
                w = WeightInitializer.normal((in_size, out_size), seed=self.seed)
            else:
                raise ValueError("Metode inisialisasi tidak valid.")
            
            self.weights.append(w)
            
    # Saatnya forward propagation
    def forward_propagation(self, input_data):
        data = input_data
        for i in range(len(self.weights)):
            print("Layer:", i+1)
            data = np.insert(data, 0, 1)
            newdata = []
            
            for j in range(len(self.weights[i])):
                print("Neuron:", j+1)
                z = np.dot(self.weights[i][j], data)
                print("Hasil dot product:", z)
                result = ActivationFunction.sigmoid(z)
                print("Hasil:", result)
                newdata.append(result)
            
            data = newdata
            print("Hasil layer:", data)
            print("\n")
        
        # Untuk debugging
        return data
            
        
    # Untuk debugging
    def debug(self):
        return self.weights[0]
        
        
        

In [ ]:
# Test Run
# y = np.array([[0], [1], [1], [0]])

model = FFNN()
# weight = model.debug()
# print(weight)
# print(weights)

X = np.array([0, 1, 1])
data = X
model.forward_propagation(data)

    

# for i in range(len(weight)) :
#     print(i + 1)
#     z = np.dot(weight[i], data)
#     print(z)

#     result = ActivationFunction.sigmoid(z)
#     print(result)

# y_pred = model.forward_propagation(X)
# print("Predictions:", y_pred)